# Robotic Arm



A [Robotic arm](https://en.wikipedia.org/wiki/Robotic_arm) is a type of mechanical arm, usually programmable, with similar characteristics of a human arm. The purpose of the robot is to replace the human in difficult and harmful tasks e.g., Industrial production, processing and manufacturing roles. Any task in which extremely precise, fast and repeatable movements are required as well as to automate the process of doing the daily task that humans typically do.

The [arm](https://www.intel.com/content/www/us/en/robotics/robotic-arm.html#:~:text=Most%20industrial%20robotic%20arms%20use,power%20systems%2C%20and%20software%20components) 
contains:
- __Motors and gears__ that allow arm to rotate/move in different directions.
- __Joints__ that consists of several joints.
- __Links__ are the rigid segments that connect the joints.
- __End-effector__ is the device that is attached to the arm's last link e.g., grippers to pick up items.
- __Sensors__, such as cameras, force sensors, or proximity sensors, which allow the arm to sense and respond to its environment.
- __Control system__ is the brain of the robotic arm, which directs the movement and operation of the arm. It is commonly run in a computer, software, and control electronics.

# Control robot arm through a mobile application
In this code [here](https://github.com/Julieanna97/Robot-Arm-Research/blob/master/Mobile%20App/Mobile_app.cpp), the __read_signals_from_mobile_app()__ function reads the signals from the mobile application to the robot arm in binary string, which is converted using __stoi__ with a base of 2. 

It sends one uint16 (__2__ bytes) at a time, encoded like this:
- bit __1__ - __SIGNAL_UP__
- bit __2__ - __SIGNAL_DOWN__
- bit __4__ - __SIGNAL_LEFT__
- bit __8__ - __SIGNAL_RIGHT__

I'm coding this using the bitwise operation (boolean algebra) to get signals in the right place. For example, the robot arm moves to the right by shifting 3 bits to the left. To switch the robot arm's position, I use bitshift and bitwise right shift. See the code below.

In [ ]:
// Define the signals to control the robot arm
const int SIGNAL_UP    = 1 << 0; // 00000001b
const int SIGNAL_DOWN  = 1 << 1; // 00000010b
const int SIGNAL_LEFT  = 1 << 2; // 00000100b
const int SIGNAL_RIGHT = 1 << 3; // 00001000b

In [ ]:
// Function to read signals from mobile app (replace with actual code)
int read_signals_from_mobile_app() {
    // For example, if the signals are sent as a binary string
    string signals_str = "1000"; // Up and right signals are active
    int signals = stoi(signals_str, nullptr, 2);
    return signals;
}

The __control_robot_arm()__ function takes an __integer argument__ that contains the signals and uses __bitwise__ operations to determine which signals are __active__. The function then uses __conditional statements__ (logical operator) to control the robot arm based on the active signals.

In [ ]:
// Function to control the robot arm based on signals
void control_robot_arm(int signals) {
    // Determine which signals are active using bitwise operations
    bool up    = (signals & SIGNAL_UP)    != 0;
    bool down  = (signals & SIGNAL_DOWN)  != 0;
    bool left  = (signals & SIGNAL_LEFT)  != 0;
    bool right = (signals & SIGNAL_RIGHT) != 0;

    // Use the signals to control the robot arm (replace with actual code)
    // For example, if there is a function called "control_robot_arm(string command)"
    // You could do something like this:
    if (up && !down && !left && !right) {
        cout << "Moving robot arm up" << endl;
    } else if (!up && down && !left && !right) {
        cout << "Moving robot arm down" << endl;
    } else if (!up && !down && left && !right) {
        cout << "Moving robot arm left" << endl;
    } else if (!up && !down && !left && right) {
        cout << "Moving robot arm right" << endl;
    } else {
        cout << "Stopping robot arm" << endl;
    }
}

In the __main()__ function, the program calls __read_signals_from_mobile_app()__ to read the signals from the mobile application and then calls __control_robot_arm()__ with the signals as an argument to control the robot arm. 

In [ ]:
int main() {
    // Read the signals from the mobile application
    int signals = read_signals_from_mobile_app();
    cout << "Signals received: " << bitset<4>(signals) << endl;

    // Control the robot arm based on the signals
    control_robot_arm(signals);

    return 0;
}

__Output:__

Signals received: 1000

Moving robot arm right

__Note:__ This is just a simple example and you would need to replace the placeholder code with actual code to read signals from a mobile application using a communication protocol such as __Bluetooth__ or __Wi-Fi__. _Additionally_, you may need to adapt this code to the specific requirements fo your robot arm and mobile application.

# Navigation using LIDAR
[Robot arms](https://www.researchgate.net/publication/330362469_Lidar_Application_for_Mapping_and_Robot_Navigation_on_Closed_Environment) use a _variety_ of sensors to be able to navigate different objects. By measuring distance to various objects to detect obstacles such as walls, furnitures etc. It provides real-time monitoring and feedback to the robot arm's control system, allowing it to adjust its movements and behavior based on the environment. It also increases the autonomy of the robot arm by allowing it to navigate and operate in complex environments without human intervention. This can be particularly useful in warehousing, logistics, or manufacturing, where robots need to operate in dynamic and unpredictable environments.

![](https://eu-images.contentstack.com/v3/assets/blt31d6b0704ba96e9d/blt4c46a271d01eb143/63abebe3d25aa95489bdd438/VelodyneLidar_DeliveryRobot_FoV.jpg)

- In order to calculate the distance to the object we can use laser pulses to bounce to the object and then back to the sensor. We can use the equation to calculate the distance based on speed and time. However, since the electromagnetic waves go back and forth, we must divide by __two__ to get the distance to the object. Here is the formula:

$$ distance = \frac{speed-of-light \times time-of-flight} {2}  $$

__Where:__
- __distance__ is the distance between the lidar sensor and the object
- the speed of __electromagnetic radiation__ is the same _as_ the _constant_ __speed of light__ in a vacuum: $$ 299,792,458m/s$$
- [__time-of-flight__](https://www.analog.com/en/applications/technology/3d-time-of-flight.html#:~:text=3D%20time%20of%20flight%20(ToF)%20is%20a%20type%20of%20scanner,from%20a%20scene%20of%20interest) is the time it _takes_ for the __laser pulse__ to travel from the lidar sensor to the object and back to the sensor. It uses a __high power optical pulses__ in durations of __nanoseconds__: 10^-9 = $$0.000000001$$

![](https://www.ednasia.com/wp-content/uploads/sites/3/2021/02/3D_ToF.jpg?w=960)

We can write a function that calculates the distance to an object from a measured time-of-flight:

Code [Here:](https://github.com/Julieanna97/Robot-Arm-Research/blob/master/LIDAR_distance/Distance_lidar.cpp)

In [ ]:
#include <iostream>
#include <cmath>

using namespace std;

int main()
{
    const double speed_of_light = 299792458.0;  // Speed of light in m/s
    double time_of_flight;                     // Time it takes for the laser pulse to travel to the object and back in seconds
    double distance;                           // Distance between the LIDAR sensor and the object in meters
    
    // Read in the measured time-of-flight in seconds from the LIDAR sensor
    cout << "Enter time-of-flight in seconds: ";
    cin >> time_of_flight;
    
    // Calculate the distance to the object using the time-of-flight and the speed of light
    distance = (speed_of_light * time_of_flight) / 2.0;
    
    // Output the calculated distance to the object in meters
    cout << "Distance to object is: " << distance << " meters" << endl;
    
    return 0;
}

Example:

30 ns (nanoseconds) = 3.0 x 10^-9 s

__Input:__ 0.000000003

__Output:__  Distance to object is: 0.449689 meters

# Conversion from A/D

![](https://ars.els-cdn.com/content/image/3-s2.0-B9780128150719000026-f02-02-9780128150719.jpg)

Many robotic arms use sensors i.e., encoders, force or vision sensors, to provide feedback about the robot's position and force.

The ADC (Analog-to-Digital) is used to convert analog sensor readings, such as precise position, velocity or force, into electrical/digital signals that can be processed by the robot's control system. The control system can then use the digital signals to adjust the robot's movements and maintain precise control. The DAC (Digital-to-Analog), is used to convert digital control signals from the robot's control system into analog control signals that can be sent to the robot's actuators with high precision.


![Digital Signal](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Digital-signal-noise.svg/1200px-Digital-signal-noise.svg.png)

![](https://cdn1.byjus.com/wp-content/uploads/2022/05/Difference-Between-Analog-And-Digital-Signal.png)

A __digital signal__ is a signal that uses binary to represent information. The information is represented as a series of __binary__ digits or __"bits"__ which is made of __1s__ and __0s__.

The analog input voltage is _compared_ to the __reference__ voltage, and the resulting voltage difference is converted into a digital value by the ADC:

$$ Digital Value = \frac{2^N \times AnalogInputVoltage} {Reference Voltage}  $$


__Where:__
- __Digital Value:__ The numerical value produced by the ADC, which is converted to a position or orientation value by the robotic arm's controller.
- __Analog Input Voltage:__ The voltage signal generated by the sensor measuring the joint position or orientation. The range of analog input voltage _depends_ on the __reference voltage__. __For example__, a temperature sensor might generate an analog voltage input that ranges from __0V to 5V__, while a pressure sensor might generate an input voltage ranging from __0V to 10V__. It's __essential__ to choose the __appropriate__ range and resolution of the analog input voltage to ensure that the signal is accurately measured and processed by the robot's control system.
- __Reference Voltage:__ The voltage used as a reference for the ADC. This is the voltage that is used to convert the analog input voltage to a digital value. This basically sets the limit of the analog input voltage range that can be measured by the ADC depending on the reference voltage. Example of range: __0V to 10V__.
- __n__: The number of bits used by the ADC. This determines the resolution of the ADC and the number of possible digital values it can produce.

In a robotic arm, the digital value produced by the ADC is typically fed into the arm's controller, which then uses this information to calculate the joint position or ortientation and determine how the arm needs to move to perform a specific task.

![adc conversion formula](https://www.rfwireless-world.com/images/n-bit-ADC-calculator.jpg)

For example:

Number of bits (n): 8

Analog voltage: 0.5V

Reference voltage: 5V

$$ Digital Value = \frac{2^8 \times 0.5} {5} = 25 $$

Digital output: __25__ (decimal) and in binary it is __11001b__

Another example:

If the _reference_ __voltage__ is 5V, and the ADC has a __8-bit__ _resolution_, the ADC can produce $$ 256(2^8)$$ possible digital values, with each value representing a voltage step of approximately $$ \frac{5}{256} = 19.53 mV (millivolts) = 0,01953125V$$ Which is equivalent to: $$ 19 \times 10^-3 = 0,019V$$

__Note:__ That means the minimum change in the input which can be detected by the ADC is equal to 0,019V. __If__ the change in the input signal is less than e.g., __1.25V__, then it won't get detected by this given ADC. On the other end, if the __full-scale range__ of the ADC is __1V__, which is the resolution in __bits__:
$$ \frac{1V}{2^8} = 0,00390625V = 3,90 mV$$ 
So now, the minimum change which can be detected by the ADC is equal to 3,90mV. So, in this way, by changing the reference voltage, the minimum detectable voltage can be increased. But at the same time, the conversion range of the ADC will also reduce. So, in a way, we can say that there is a trade-off for changing the reference voltage of the given ADC. But keeping the same reference voltage, by increasing the number of bits, we can increase the resolution (the amount of signal we want it to receive which can be fixed or manipulated).

__Note:__ The choice of reference voltage depends on the specific application and the requirements for accuracy and resolution. It's essential to choose a reference voltage that is stable and accurate to ensure that the ADC produces reliable and accurate measurements.

In order for me to measure the __volt__ _per_ second or nanosecond, I need to measure the frequency of the analog signal, in the _unit_ __Hertz(Hz)__, which is a unit of frequency that measures the number of cycles of a periodic wave per second. We can express it __as__:
$$ f = \frac{2 cycle}{0.5seconds} = 2Hz$$

where __f__ is the number of cycles that occur in __1__ second and the frequency is measured in __Hz__ and __seconds__ is the time it takes for the signal to complete one cycle, and it is measured in seconds.

In this code, I define the ADC _voltage_ reference, resolution and sampling rate in Hz, as well as the length of the robotic arm, as constants.

I then read an ADC value, which would come from an ADC sensor on the robotic arm. Calculate the voltage step per second and per nanosecond(10^-9) using the formula: $$ adc voltage step per second = \frac {Vref} {2^N} \times fs $$

Vref: voltage reference

N: ADC resolution (in bits)

fs: ADC sampling rate (frequency in Hz)

Next, I convert the ADC value to a voltage using the formula:
$$ voltage = \frac {adc value}{2^N} \times Vref $$

Then we calculate the angle of the robotic arm using the formula:
$$ angle = asin(\frac{voltage}{L}) $$

$$ 1 nanosecond = 1.0E-9 seconds = 0,000000001 seconds $$
Here is an example [code](https://github.com/Julieanna97/Robot-Arm-Research/blob/master/ADC_Voltage/ADC_Volt_Measure.cpp):

In [ ]:
#include <iostream>
#include <cmath>

// ADC voltage reference in volts
const double ADC_REFERENCE_VOLTAGE = 5.0;

// ADC resolution in bits
const int ADC_RESOLUTION = 12;

// ADC sampling rate in Hz
const int ADC_SAMPLING_RATE = 100000;

// Length of the robotic arm in meters
const double ROBOTIC_ARM_LENGTH = 1.0;

int main()
{
    // Read ADC value, it can be volts also
    int adc_value = 1234; // Replace with actual ADC reading

    // Calculate the voltage step per second
    double adc_voltage_step_per_second = ADC_REFERENCE_VOLTAGE / pow(2, ADC_RESOLUTION) * ADC_SAMPLING_RATE;

    // Calculate the voltage step per nanosecond
    double adc_voltage_step_per_nanosecond = adc_voltage_step_per_second / 1e9;

    // Calculate the frequency in Hertz
    double frequency = ADC_SAMPLING_RATE / pow(2, ADC_RESOLUTION);

    // Convert ADC value to voltage
    double voltage = adc_value / pow(2, ADC_RESOLUTION) * ADC_REFERENCE_VOLTAGE;

    // Calculate the angle of the robotic arm
    double angle = asin(voltage / ROBOTIC_ARM_LENGTH);

    // Output the results
    std::cout << "ADC voltage step per second: " << adc_voltage_step_per_second << " V/s" << std::endl;
    std::cout << "ADC voltage step per nanosecond: " << adc_voltage_step_per_nanosecond << " V/ns" << std::endl;
    std::cout << "ADC frequency: " << frequency << " Hz" << std::endl;
    std::cout << "ADC value: " << adc_value << std::endl;
    std::cout << "Voltage: " << voltage << " V" << std::endl;
    std::cout << "Robotic arm angle: " << angle << " radians" << std::endl;

    return 0;
}


__Output:__

ADC voltage step per second: 122.07 V/s

ADC voltage step per nanosecond: 1.2207e-007 V/ns

ADC frequency: 24.4141 Hz

ADC value: 1234

Voltage: 1.50635 V

Robotic arm angle: nan radians

# Energy Consumption

The [electricity consumption](https://www.energuide.be/en/questions-answers/how-can-i-calculate-the-consumption-of-an-electrical-appliance/94/) of a robotic arm _depends_ on __several factors__:
- the __power rating__ of the __motors and control systems__
- the _regulation_ of the system, and the duration of the __program/operation__

![energy consumption](https://www.mdpi.com/energies/energies-12-00027/article_deploy/html/images/energies-12-00027-g013.png)

__Where:__
- __Power rating:__ The rated power of the robotic arm's motors and control systems, typically measured in __watts (W)__ or __kilo__ _watts_ __(kW).__
- __Time:__ the duration of operation, typically measured in __hours (h).__

## During __idle__

The _formula_ for calculating the electricity consumption of a __robotic arm__ at __rest__ is:
$$ E = Power rating \times Time $$

__Example:__

A __power rating__ of __100 W__ and is at __rest__ for __10 hours__, the electricity consumption would be:
$$ E = 100W \times 10h = 1000Wh = 1kWh $$

__Note:__ this formula assumes that the robotic arm is not performing any tasks and is simply in a __standby or idle mode__. If the robotic arm is actively performing tasks, the electricity consumption will depend on the type of tasks being performed and the efficiency of the system.

## During __operation__
The _formula_ for calculating the electricity consumption of a __robotic arm__ when it is __operating__ is:
$$ E = Power rating \times Time \times Efficiency $$

__Where:__
- __Efficiency:__ of the robotic arm system, which is the __ratio__ of useful work performed by the arm to the total __energy__ _input_. Efficiency is typically _expressed_ as a __percentage__.

__Example:__

A __power rating__ of __1000W (1kWh)__ and is working for 2 hours with an efficiency of __80%__, the electricity consumption would be:
$$ E = 1000W \times 2 h \times 0.8 = 1600 Wh = 1.6 kWh $$

__Note:__ this formula is a simple version, and the actual electricity consumption of a robotic arm during operation _may_ __vary depending__ on the specific design and operation of the arm.

## Robot's battery span

# Statistics on Energy Consumption

I will be __using standard deviation__ for EC to gain insight into the variability of energy usage and identify areas for improvement to optimize the robot arm's energy efficiency.

- __Note:__ Make sure to __GATHER__ data for the same type of __task or operation__ performed by the robot arm during trial. This __DOES NOT__ apply for different robots as each robot has unique tasks and different __requirements__ in _order_ to function __PROPERLY.__

__1st step__ is to look at the power consumption of each joint, aka.
![](https://www.mdpi.com/applsci/applsci-10-02223/article_deploy/html/images/applsci-10-02223-g001.png)
including sensors, controllers, etc. This information can typically be found in the __manufacturer's specifications__ or _through_ __testing__. Other factors that contribute to energy consumption are:
- __payload, velocity, acceleration and temperature__
- For example, researchers found that along with payload (the weight the robot can lift), energy consumption increased from 2 to 10 Joules.
- There's many factors that can contribute to increased EC. Which is why it's essential to do study on statistics to compare dozens of robots. By taking this into consideration, the robot's movements can e.g., be smoothen out to minimise acceleration and optimise energy consumption etc.

__2nd step__ is to calculate the total power consumption.

__3rd step__ is to test how long the robot arm will operate.

__4th step__ Once the total power consumption and operating time have been figured, multiply the 2 values together to calculate the total energy consumption. The __units__ for __energy consumption__ are usually measured in __watt-hours(Wh).__

### During operation: 
__For example__, let's say I have collected __data__ of __10__ robot arm in the same manufacture to see the average value of energy consumption _within_ a __week over 8 hours__ of __operation__ _period._ With energy input to __80%.__

Formula for calculating energy consumption during operation is:
$$ 2000W \times 8 hours \times 0.8 = 12,800 W = 12,8kWh $$

### Standard deviation

1. When we need to calculate the _standard deviation_ we start by calculating the mean value of the energy consumption by adding up all the data points and divide by the total number of data points.

__EC Values:__
$$ 2000, 1500, 3000, 4500, 3200, 1700, 5000, 6000, 4500, 3600 Watts$$

2. __Calculate__ the _mean_ by adding up all the data values and dividing the total number of data points. In this example we _divide_ by __10.__

$$ \frac {2000, 1500, 3000, 4500, 3200, 1700, 5000, 6000, 4500, 3600 Watts} {10}$$

$$ Mean value(ms) = 3500W = 3,5 kWh $$

3. Now we can calculate the deviation from the mean for each of the __EC__ observation _values._

__EC Table deviation:__


<!-- Energy consumption table -->
| $$Robot Arm Model 1$$| $$m_s = 3500$$ |  |
| -----| ------| ------|
| $$x_s$$ | $$x_s - m_s$$ |
| $$1500$$ | $$-2000$$ |
| $$1700$$ | $$-1800$$ |
| $$2000$$ | $$-1500$$ |
| $$3000$$ | $$-500$$ |
| $$3200$$ | $$-300$$ |
| $$3600$$ | $$100$$ |
| $$4500$$ | $$1000$$ |
| $$4500$$ | $$1000$$ |
| $$5000$$ | $$1500$$ |
| $$6000$$ | $$2500$$ |

# Problem 1: Accuracy

The accuracy of a robot arm in terms of position and orientation refers to how precisely the arm can move to a specific location and how accurately it can orient its end-effector in a particular direction. This accuracy is typically measured repeatedly along with its positional accuracy.

The accuracy issue is based on the object's displacement from the target area after a series of repeated movements by the arm. It could be error in accuracy along the X or Y-axis due to sensor errors, design, mechanical wear and tear, control algorithms etc.

We can measure the error by:
- [Euclidean system](https://www.kth.se/polopoly_fs/1.1078075.1622113235!/gr22rapport2021-02.pdf) by comparing the __position__ and __orientation__ of the end-effector to the target position and orientation to see the _displacement_ in both __x, y & z__ directions.

![Model](https://i.pcmag.com/imagery/encyclopedia-terms/x-y-z-matrix-xymatrix.fit_lim.size_1050x.gif)

measured along:

x = eastwest axis

y = northsouth axis

z = height/elevation

Calculation for Eculidean distance between each target point and the measure point:
$$ AP_p = {\sqrt[2]{(\bar{x1}-x2_c)^2+(\bar{y1}-y2_c)^2+(\bar{z1}-z2_c)^2}} $$

Code [here](https://github.com/Julieanna97/Robot-Arm-Research/blob/master/Accuracy%20measurement/Accuracy_Euclidean.cpp)

Example of table _positions_ of x, y & z:

<!-- Target Position Table -->
| $X-Target$ | $Y-Target$ | $Z-Target$ |
| -----| ------| ------|
| $X1 = 10.0$ | $Y1 = 6.0$ | $Z1 = 4.0$ |
-
<!-- Starting Position Table -->
| $X-Measured$ | $Y-Measured$ | $Z-Measured$ |
| -----| ------| ------|
| $X2 = 9.8$ | $Y2 = 6.1$ | $Z2 = 4.2$ |

Where $\bar{x1}-x2_c, \bar{y1}-y2_c, \bar{z1}-z2_c$ is the error along the x, y and z-axis.

$ Calculation: $
$$ AP_p = {\sqrt[2]{(\bar{10.0}-9.8_c)^2+(\bar{6.0}-6.1_c)^2+(\bar{4.0}-4.2_c)^2}} $$

We get the following value:

$$ 0.04 + 0.01 + 0.04 = 0,09$$

Square root of 2 = $${\sqrt[2]{0,09}} => 0.3$$

To calculate the average/mean distance as a measure of the accuracy of the robotic arm:
$$ m = \frac{sum}{distance}$$

Example:
sum = 4.5
$$ m = \frac{4.5}{0.3} = 15 $$
__X & Y__ can be measured in __millimeters__, __centimeters__ or __inches__, depending on the application and the precision __required__. Robotic arms are required to be __highly__ precise, with __accuracy__ levels measured in _millimeter_ or even _microns_.

Lastly, we calculate the accuracy error as the difference between the average distance and the ideal distance of e.g., __7.0__ using absolute.
$$ abs(15 - 7.0) $$
So the __accuracy error__ is: __8__ _mm_